In [1]:
import cv2
import os
import numpy as np 
import matplotlib.pyplot as plt 

from tqdm import tqdm
import mlflow

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score

## MLFlow Initialization

In [2]:
mlflow.set_tracking_uri("file:///home/user/MLFlow2/")

In [3]:
experiment = "Context-to-PixelFusion"

try:
    idExperiment = mlflow.create_experiment(experiment)
except:
    idExperiment = mlflow.get_experiment_by_name(experiment).experiment_id

## Define functions

In [4]:
def calculate_metrics(TP, TN, FP, FN):
    
    sensitivity = TP / (TP + FN)
    
    specificity = TN / (TN + FP)
    
    accuracy = (TP + TN) / (TP+FN+FP+TN)
    
    recall= TP / (TP+FN)
    
    precision = TP/(TP + FP)
    
    baccuracy = (sensitivity + specificity) / 2
    
    IOU = TP / (TP + FP + FN)
    
    F1_score = 2 * ((precision * recall) / (precision + recall))
    

    metrics = {'iou': IOU, 'f1_score':F1_score,'accuracy':accuracy,'balanced_accuracy':baccuracy, 'precision': precision, 'recall': recall}
    return metrics

In [37]:
def log_metrics(input_path, filter_mask_path, groundtruth_path, output_path):
    
    print(input_path)
    print(filter_mask_path)
    tp = tn = fp = fn = 0
    kernel = np.ones((3, 3), np.uint8)

    for filename in os.listdir(input_path):

        mask = cv2.imread(os.path.join(input_path, filename), cv2.IMREAD_GRAYSCALE)
        mask = cv2.dilate(mask, kernel, iterations=1)
        groundtruth = cv2.imread(os.path.join(groundtruth_path, filename),cv2.IMREAD_GRAYSCALE)
        groundtruth = cv2.dilate(groundtruth, kernel, iterations=1)

        tn_temp, fp_temp, fn_temp, tp_temp = confusion_matrix(groundtruth.flatten(), mask.flatten()).ravel()

        tp += tp_temp
        tn += tn_temp
        fp += fp_temp
        fn += fn_temp

    metrics = calculate_metrics(tp, tn, fp, fn)

    ################ Apply Filter of Model Classification ################
    tp = tn = fp = fn = 0
    
    if not os.path.isdir(output_path):
        os.makedirs(output_path)

    for filename in os.listdir(input_path):

        mask = cv2.imread(os.path.join(input_path, filename), cv2.IMREAD_GRAYSCALE)
        mask = cv2.dilate(mask, kernel, iterations=1)
        groundtruth = cv2.imread(os.path.join(groundtruth_path, filename),cv2.IMREAD_GRAYSCALE)
        groundtruth = cv2.dilate(groundtruth, kernel, iterations=1)
        mask_filter = cv2.imread(os.path.join(filter_mask_path, "mask/"+filename), cv2.IMREAD_GRAYSCALE)
        
        print(mask.shape)
        print(mask_filter.shape)
        mask_filtered = cv2.bitwise_and(mask, mask_filter)
        cv2.imwrite(os.path.join(output_path, filename), mask_filtered)
           
        c_m = confusion_matrix(groundtruth.flatten(), mask_filtered.flatten(), labels = [0, 255])
        '''if c_m.shape[0] == 1 and c_m.shape[1] == 1:
            tn_temp, fp_temp, fn_temp, tp_temp = c_m.ravel()[0], 0 , 0 , 0
        else:'''
        tn_temp, fp_temp, fn_temp, tp_temp = c_m.ravel()

        #tn_temp, fp_temp, fn_temp, tp_temp = np.float64(tn_temp), np.float64(fp_temp), np.float64(fn_temp), np.float64(tp_temp)

        tp += tp_temp
        tn += tn_temp
        fp += fp_temp
        fn += fn_temp

    metrics_filtered = calculate_metrics(tp, tn, fp, fn)


    with mlflow.start_run(experiment_id=idExperiment, run_name=input_path.split("/")[1]+"_"+filter_mask_path.split("/")[1]):
        mlflow.log_param("MODEL_SEGMENTATION", input_path.split("/")[1])
        mlflow.log_param("MODEL_CLASSIFICATION", filter_mask_path.split("/")[1])
        print(metrics)
        print(metrics_filtered)
        #mlflow.log_artifact("/content/image_final.png")
        for key in metrics:
            mlflow.log_metric("TEST_"+key, metrics[key])
        for key in metrics_filtered:
            mlflow.log_metric("TEST_"+key+"_FILTERED", metrics_filtered[key])


## Parameters Initialization

In [38]:
OUTPUT_DIR_PRE = "./output/PRE/"
OUTPUT_DIR_CRI = "./output/CRI/"
OUTPUT_DIR_CPF = "./output/CPF/"

DATA_DIR = "./amazonwildroadsdataset/"
groundtruth_path = os.path.join(DATA_DIR, "mask")

## Fusion and log metrics

In [39]:
list_results_PRE = os.listdir(OUTPUT_DIR_PRE)
list_results_CRI = os.listdir(OUTPUT_DIR_CRI)

In [40]:
list_results_class

['resnet']

In [41]:
if not os.path.isdir(OUTPUT_DIR_CPF):
    os.makedirs (OUTPUT_DIR_CPF)

In [46]:
for result_PRE in list_results_PRE:
    for result_CRI in list_results_CRI:
        output_join_path = os.path.join(OUTPUT_DIR_CPF, result_PRE+ "_"+ result_CRI)
        input_path = os.path.join(OUTPUT_DIR_PRE, result_PRE)
        
        filter_mask_path = os.path.join(OUTPUT_DIR_CRI, result_CRI)
        log_metrics(input_path, filter_mask_path, groundtruth_path, output_join_path)

./output/PRE/UnetPlusPlus-timm-efficientnet-b7-imagenet-all_patches
./output/CRI/resnet
(1072, 2792)
(1072, 2792)
(1059, 2793)
(1059, 2793)
(1059, 2794)
(1059, 2794)
(1059, 2794)
(1059, 2794)
(1059, 2794)
(1059, 2794)
(1059, 2794)
(1059, 2794)
(1059, 2793)
(1059, 2793)
(1059, 2793)
(1059, 2793)
{'iou': 0.5317351010608061, 'f1_score': 0.6942912004726561, 'accuracy': 0.9930141079343066, 'balanced_accuracy': 0.8746774895156499, 'precision': 0.6434932303859159, 'recall': 0.7537966259341223}
{'iou': 0.44933468551137146, 'f1_score': 0.6200564852318177, 'accuracy': 0.9924170216999415, 'balanced_accuracy': 0.7923412588885888, 'precision': 0.6558547841797399, 'recall': 0.5879638538760062}
